# A Visual Notebook to Using BERT for the First Time.ipynb



<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" />

In this notebook, we will use a pre-trained deep learning model to process some text. We will then use the output of that model to classify the text. The text is a list of sentences from film reviews. And we will classify each sentence as either speaking "positively" about its subject or "negatively".

## Models: Sentence Sentiment Classification
Our goal is to create a model that takes a sentence (just like the ones in our dataset) and produces either 1 (indicating the sentence carries a positive sentiment) or a 0 (indicating the sentence carries a negative sentiment). We can think of it as looking like this:

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Dataset
The dataset we will use in this example is [SST2](https://nlp.stanford.edu/sentiment/index.html), which contains sentences from movie reviews, each labeled as either positive (has the value 1) or negative (has the value 0):


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>

## Installing the transformers library
Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 2.7MB/s 
     |████████████████████████████████| 2.9MB 17.5MB/s 
     |████████████████████████████████| 890kB 30.8MB/s 
     |████████████████████████████████| 1.1MB 35.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=84d81e7dee5f76c973396480663300c930e76f332405c826f2ef5b03bbfb8640
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
# import warnings
# warnings.filterwarnings('ignore')

##Importing the Kaggle Dataset

In [110]:
# from: https://www.kaggle.com/general/74235
!pip install kaggle
!pip update 
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Change permisions of the file
!chmod 600 ~/.kaggle/kaggle.json
# following command from: https://www.kaggle.com/c/nlp-getting-started/data
!kaggle competitions download -c nlp-getting-started

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 56.2MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 24.1MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.7MB/s]


In [139]:
# Load in training data
train_df_full = pd.read_csv('/content/train.csv')
train_df = train_df_full[['text','target']]
print(train_df)
batch_1 = train_df[:2000]
print("How much of each?\n", batch_1["target"].value_counts())

                                                   text  target
0     Our Deeds are the Reason of this #earthquake M...       1
1                Forest fire near La Ronge Sask. Canada       1
2     All residents asked to 'shelter in place' are ...       1
3     13,000 people receive #wildfires evacuation or...       1
4     Just got sent this photo from Ruby #Alaska as ...       1
...                                                 ...     ...
7608  Two giant cranes holding a bridge collapse int...       1
7609  @aria_ahrary @TheTawniest The out of control w...       1
7610  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...       1
7611  Police investigating after an e-bike collided ...       1
7612  The Latest: More Homes Razed by Northern Calif...       1

[7613 rows x 2 columns]
How much of each?
 0    1246
1     754
Name: target, dtype: int64


## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [6]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


For performance reasons, we'll only use 2,000 sentences from the dataset

In [124]:
batch_1 = df[:2000]
type(batch_1)

pandas.core.frame.DataFrame

We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [5]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [140]:
# For DistilBERT:
model_class = ppb.DistilBertModel
tokenizer_class = ppb.DistilBertTokenizer
pretrained_weights = 'distilbert-base-uncased'

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [141]:
print('Tokenizer:', type(tokenizer))
print('Model:', type(model))

Tokenizer: <class 'transformers.tokenization_distilbert.DistilBertTokenizer'>
Model: <class 'transformers.modeling_distilbert.DistilBertModel'>


Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [143]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [144]:
print('tokenized is of type:', type(tokenized))
print(tokenized)

tokenized is of type: <class 'pandas.core.series.Series'>
0       [101, 2256, 15616, 2024, 1996, 3114, 1997, 202...
1       [101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...
2       [101, 2035, 3901, 2356, 2000, 1005, 7713, 1999...
3       [101, 2410, 1010, 2199, 2111, 4374, 1001, 3748...
4       [101, 2074, 2288, 2741, 2023, 6302, 2013, 1009...
                              ...                        
1995    [101, 4040, 4053, 2988, 1999, 2225, 5298, 8299...
1996    [101, 4931, 1996, 1001, 15426, 2293, 2725, 405...
1997    [101, 1030, 2852, 14573, 3567, 4063, 1030, 464...
1998    [101, 1005, 17454, 2015, 1997, 8867, 5725, 101...
1999    [101, 1030, 8981, 10024, 2094, 22610, 3398, 20...
Name: text, Length: 2000, dtype: object


<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [145]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print('Maximum sentence length:', max_len)

# # From https://stackoverflow.com/a/50339260/5106570
# measurer = np.vectorize(len)
# res1 = measurer(tokenized).max(axis=0)
# print('max length:', res1)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('Padded numpy array:\n', padded)

Maximum sentence length: 82
Padded numpy array:
 [[  101  2256 15616 ...     0     0     0]
 [  101  3224  2543 ...     0     0     0]
 [  101  2035  3901 ...     0     0     0]
 ...
 [  101  1030  2852 ...     0     0     0]
 [  101  1005 17454 ...     0     0     0]
 [  101  1030  8981 ...     0     0     0]]


Our dataset is now in the `padded` variable, we can view its dimensions below:

In [146]:
np.array(padded).shape

(2000, 82)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [147]:
attention_mask = np.where(padded != 0, 1, 0) # if padded != 0 then element is 1 otherwise 0
print('Shape of attention mask:', attention_mask.shape)
print('Attention mask looks like this:\n', attention_mask)

Shape of attention mask: (2000, 82)
Attention mask looks like this:
 [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [148]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    print('Type, Value:', (type(last_hidden_states), last_hidden_states))

Type, Value: (<class 'tuple'>, (tensor([[[-0.0078,  0.2434, -0.1924,  ..., -0.0423,  0.5865,  0.0078],
         [ 0.3320,  0.3346,  0.0207,  ..., -0.1569,  0.8014, -0.4104],
         [ 0.5480,  0.2635,  0.1367,  ...,  0.0184,  0.0848, -0.2652],
         ...,
         [ 0.0152,  0.2166,  0.1225,  ...,  0.1965,  0.0519, -0.1487],
         [-0.0369,  0.2852,  0.1945,  ...,  0.0811,  0.0711, -0.1032],
         [ 0.0386,  0.2874,  0.1401,  ...,  0.0651,  0.1448, -0.0140]],

        [[-0.5075,  0.1034, -0.6453,  ..., -0.2546,  0.2126,  0.3391],
         [ 0.4142,  0.5571, -0.3625,  ..., -0.0183,  0.3380, -0.0851],
         [ 0.3048,  0.2969, -0.3315,  ..., -0.3804, -0.0696, -0.3818],
         ...,
         [-0.1041,  0.1329, -0.2229,  ..., -0.0524, -0.0426,  0.2601],
         [-0.0437,  0.0847, -0.2386,  ...,  0.0112, -0.1759,  0.4035],
         [ 0.0222,  0.1052, -0.3387,  ...,  0.0053, -0.2005,  0.2808]],

        [[-0.1455, -0.1657,  0.2994,  ..., -0.1828,  0.0257,  0.4130],
         [-0.

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [149]:
features = last_hidden_states[0][:,0,:].numpy()

In [150]:
print(features)

[[-0.00779025  0.24342766 -0.19241537 ... -0.04230952  0.58648366
   0.00777953]
 [-0.507526    0.10338616 -0.6452901  ... -0.2545608   0.21263906
   0.33910495]
 [-0.14549513 -0.16572241  0.29940492 ... -0.18275894  0.02567799
   0.41298452]
 ...
 [-0.25057685 -0.11841887  0.0774207  ... -0.05480941  0.15326285
   0.28441513]
 [ 0.00270303 -0.03092067 -0.00259606 ... -0.02553035  0.22686933
   0.24538912]
 [-0.04196359  0.12659155  0.09899823 ... -0.16992418  0.24939045
   0.476121  ]]


The labels indicating which sentence is positive and negative now go into the `labels` variable

In [152]:
labels = batch_1['target']
print(labels)

0       1
1       1
2       1
3       1
4       1
       ..
1995    1
1996    1
1997    1
1998    0
1999    1
Name: target, Length: 2000, dtype: int64


## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [153]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />

### [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [154]:
lr_clf = LogisticRegression(random_state=0, max_iter=330) #had to adjust max_iter from default 100 to 330, about the edge of no convergence
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=330,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating F1 score for prediction on test set

In [52]:
# Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
from sklearn.metrics import f1_score
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
print(f1_score(y_true, y_pred, average='macro')) # should be 0.26...
print(f1_score(y_true, y_pred, average='micro')) # should be 0.33...
print(f1_score(y_true, y_pred, average='weighted')) # should be 0.26...
print(f1_score(y_true, y_pred, average=None)) # should be array([0.8, 0. , 0. ])
y_true = [0, 0, 0, 0, 0, 0]
y_pred = [0, 0, 0, 0, 0, 0]
print(f1_score(y_true, y_pred, zero_division=1))

0.26666666666666666
0.3333333333333333
0.26666666666666666
[0.8 0.  0. ]
1.0


In [156]:
y_pred = lr_clf.predict(test_features)
y_true = list(test_labels)
print('y_pred:', list(y_pred))
print('y_true:', y_true)
print(f1_score(y_true, y_pred, average='binary'))

y_pred: [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0


## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [56]:
lr_clf.score(test_features, test_labels)

0.822

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [51]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.501 (+/- 0.06)


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.